In [6]:
import pandas as pd
import io
import requests

In [7]:
# The year each user joined and current status of membership
YearJoined = pd.read_csv("year_joined.csv") 
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [8]:
# Number of emails you sent out in a given week that were opend by the member
Emails = pd.read_csv("emails.csv")
Emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [39]:
# TIme a memeber donated to your organization
Donations = pd.read_csv("donations.csv")
Donations.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


In [10]:
# Total Number of Users
YearJoined.user.count()

1000

In [11]:
# Checking if there are multiple entries for user status for each year
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [12]:
# Are null weeks reported? 
Emails[Emails.emailsOpened < 1]
# It seems like the nulls are not reported or members always have at least one email event
# However, is it likely that people always open their emails (at least one email) every week?
# We can look at a specitif user

,emailsOpened,user,week


In [13]:
Emails[Emails.user == 5] # Opening Email History for User 5

,emailsOpened,user,week
147,1.0,5.0,2017-06-05 00:00:00
148,1.0,5.0,2017-06-12 00:00:00
149,1.0,5.0,2017-06-19 00:00:00
150,2.0,5.0,2017-06-26 00:00:00
151,1.0,5.0,2017-07-03 00:00:00
152,1.0,5.0,2017-07-17 00:00:00
153,2.0,5.0,2017-07-24 00:00:00
154,2.0,5.0,2017-07-31 00:00:00
155,2.0,5.0,2017-08-07 00:00:00
156,2.0,5.0,2017-08-14 00:00:00


In [14]:
Emails[Emails.user == 5].shape # 46 Number of Entries 

(46, 3)

In [15]:
# How many weeks in between the first and the last reported entries = 51 entries = 52 weeks
# which means we are missing some weeks of data (6 weeks are missing)
import datetime
date_time_str_max = max(Emails[Emails.user == 5].week)
date_time_str_min = min(Emails[Emails.user == 5].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

51.0

In [16]:
# filling in missing data! missing week = 0
complete_idx = pd.MultiIndex.from_product((set(Emails.week), set(Emails.user)))
complete_idx

MultiIndex([('2015-07-27 00:00:00',   1.0),
            ('2015-07-27 00:00:00',   3.0),
            ('2015-07-27 00:00:00',   5.0),
            ('2015-07-27 00:00:00',   6.0),
            ('2015-07-27 00:00:00',   9.0),
            ('2015-07-27 00:00:00',  10.0),
            ('2015-07-27 00:00:00',  14.0),
            ('2015-07-27 00:00:00',  16.0),
            ('2015-07-27 00:00:00',  20.0),
            ('2015-07-27 00:00:00',  21.0),
            ...
            ('2015-07-13 00:00:00', 973.0),
            ('2015-07-13 00:00:00', 977.0),
            ('2015-07-13 00:00:00', 982.0),
            ('2015-07-13 00:00:00', 984.0),
            ('2015-07-13 00:00:00', 987.0),
            ('2015-07-13 00:00:00', 991.0),
            ('2015-07-13 00:00:00', 992.0),
            ('2015-07-13 00:00:00', 993.0),
            ('2015-07-13 00:00:00', 995.0),
            ('2015-07-13 00:00:00', 998.0)],
           length=93247)

In [17]:
all_email = Emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [18]:
all_email[all_email.user == 5].sort_values('week')

,week,user,emailsOpened
86781,2015-02-09 00:00:00,5.0,0.0
81391,2015-02-16 00:00:00,5.0,0.0
35576,2015-02-23 00:00:00,5.0,0.0
81930,2015-03-02 00:00:00,5.0,0.0
42583,2015-03-09 00:00:00,5.0,0.0
...,...,...,...
61987,2018-04-30 00:00:00,5.0,2.0
33420,2018-05-07 00:00:00,5.0,1.0
4853,2018-05-14 00:00:00,5.0,1.0
91093,2018-05-21 00:00:00,5.0,2.0


In [19]:
# page 32
cutoff_dates = Emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index(drop=True)
cutoff_dates

,user,min,max
0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00
...,...,...,...
534,991.0,2016-10-24 00:00:00,2016-10-24 00:00:00
535,992.0,2015-02-09 00:00:00,2015-07-06 00:00:00
536,993.0,2017-09-11 00:00:00,2018-05-28 00:00:00
537,995.0,2016-09-05 00:00:00,2018-05-28 00:00:00


In [20]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

In [21]:
all_email[all_email.user == 5].shape

(52, 3)

In [37]:
Donations.head()

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0


In [34]:
# Relating email data to donations data
# We want to downsample the donation data (turn it into weekly time series)
# Interested in the total weekly donation for each user

# converting string timestamp to proper timestamped data calss
Donations.timestamp = pd.to_datetime(Donations.timestamp)
Donations.set_index('timestamp', inplace=True) #setting index as each week
agg_don = Donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())
# apply(lamda df: df.amount.action verbs applies the action verbs equally to all entries in df)
# for each user, sum donations by week, drop weeks without donation, 
# "W-MON" is to resample with anchored week-week achored to Monday- to match the same weekly dates 

In [47]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

In [45]:
merged_df = pd.DataFrame() # Create a dataframe

for user, user_email in all_email.groupby('user'):
  user_donations = agg_don[agg_don.index.get_level_values('user') == user] 
  #

  user_donations = user_donations.droplevel(0) #drop weeks without donation
  # user_donations.set_index('timestamp', inplace = True)  
  
  user_email = all_email[all_email.user == user]
  user_email.sort_values('week', inplace=True)
  user_email.set_index('week', inplace=True)

  df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
  df.fillna(0)

  merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [46]:
merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN
